# Project 2 Jupyter Notebook
## by Qingyue Li

## Scientific Question: Would the storage protein, an allergen, in peanut induce allergenic cross-reactive with pistachio as cashew and pistachio have high cross-reaticity for one another?
    
## Background:
    Storage protein in seed or nuts are responsible to most of the allergenic reactions. There are many studies that are intersted in the cross reactivity between these allergens. Cross-reactivity in terms of allergic reaction is the likely hood of being allergic to another source when one is already allergic to a known source. For example, in this project, we are trying to figure out if someone is allergy to pistachio, would they more likely to be allergic to cashew or peanuts. 
    Allergy is caused by the specific binding between the allergen and the IgE antibody. If the conformation of a protein contains a IgE-binding epitope, then this protein would be highly allergenic. So when two proteins express high similarity at the molecular level, then they are very likely to cross-react.


## Hypothesis: If the seed storage proteins (Ana o 2) in cashew and pistachio (2S Albumin) have multiple motifs in sequence and similar protein residue (binding sites), then they are more likely to have realitively higher cross-reactivity with one another than with peanuts.

## Method
    Attempted protein structure analysis to see the similarity of the allergenic proteins, such as amino acids component, the hydrophobicity (as it is important for binding site interaction). Pairwise sequence alignment is performed to see how similar the sequences are and a heat plot is generated for visulization.

Arachis hypogaea (peanut)
Arachis hypogaea main allergen Ara h1 mRNA, complete cds
https://www.ncbi.nlm.nih.gov/nuccore/193850560
crystal structure: https://www.rcsb.org/structure/3s7i

Pistacia vera (pistachio)
Pistacia vera Pis v 1 allergen 2S albumin mRNA, complete cds
https://www.ncbi.nlm.nih.gov/nuccore/110349080

Anacardium occidentale (cashew)
Anacardium occidentale allergen Ana o 2 mRNA, partial cds
https://www.ncbi.nlm.nih.gov/nuccore/25991542

## Part 1: Package Loading

### SeqIO: 
    Allows the inputting and the outputting of sequence files. There are a variety of types of file that SeIO could read (but they need to be a SeqRecord object). Records of the file could be created in memory and the records could be indexed. It outputs files so that the results could be stored for reading and sharing. To learn more, visit: https://biopython.org/wiki/SeqIO.

### NumPy:
    This package includes a lot of mathematical equation. It is mainly used for performing mathematical operations. With the Numpy package, computations such as simple math operation, linear algebra, trigonomitry, and shape manipulation can be achieved. Numpy usually produces an array that contains objects of the type. To learn more, visit: https://numpy.org/doc/stable/reference/arrays.html.

### matplotlib.pyplot:
    This package contains functions that could create plots in Python. For example, x-axis and the y-axis of a plot can be defined and the type of graph can be customized as well. Some of the commonly used types of plot that could be done using the functions in matplotlib.pyplot are histogram and scatter plot. It is a great package for visualization. To learn more, visit: https://matplotlib.org/stable/tutorials/introductory/pyplot.html.

### pairwise2:
    Pairwise2 is a package from the BioPython library. This would perform a pairwise sequence alignment when called. Either global or local alignment could be choosen with its appropriate match parameter and gap penalty parameter. A positive match score indicates high compatability while negative or zero means that the sequences are imcompatible. To learn more, visit:https://biopython.org/docs/1.75/api/Bio.pairwise2.html

### nglview:
    This package read in files of certain type such as PDB amd CIF format. This is a package that allows interactive visualization of molecular structures. The user can perform tasks such as scaling and rotating. The trajectories of molecule dynamics can also be viewed. To learn more, visit: http://nglviewer.org/nglview/release/v0.5.1/#
    
### ipywidgets:
    By importing this package, a simple widget is installed. A widget is a Python object that allows interactive control of the data. It is a graphic user interface (GUI) such as the slidebar and buttons that we see on a website. This allows the modification of the outputs without having to change the code that led to the result. To learn more, visit: https://ipywidgets.readthedocs.io/en/latest/user_install.html.
    

In [ ]:
from Bio.PDB import *
import nglview as nv
import ipywidgets
from Bio.SeqUtils.ProtParam import ProteinAnalysis

from Bio import SeqIO
import numpy as np
import matplotlib.pyplot as plt
from Bio import pairwise2

### Protein Structure Analysis

In [ ]:
# To parse PDB file and create an interactive visualization
pdb_parser = PDBParser()
structure = pdb_parser.get_structure("PHA-L", "Ara_h1.pdb")
view = nv.show_biopython(structure)

# Header
# To gain infomation on a protein by creating a dictionary
mmcif_dict = MMCIFDict.MMCIFDict("Ara_h1.pdb") #change file name and type
len(mmcif_dict)

# .get_residues() method in a loop
# returns the protein sequence of the residue
for model in structure:
    for residue in model.get_residues():
        print(residue)

# Polypeptide construction
# Generating each individual poplypeptide protein sequence based on the residue sequence above

polypeptide_builder = CaPPBuilder()
counter = 1
for polypeptide in polypeptide_builder.build_peptides(structure):
    seq = polypeptide.get_sequence()
    print(f"Sequence: {counter}, Length: {len(seq)}")
    print(seq)
    counter += 1
    
    
# empty list
all_seqs = []
counter = 1

# Analyze the protein sequence using the imported ProteinAnalysis
# Variable seq from the above .get_sequence() needs to be transformed inti a str() object
# ProteinAnalysis allows more properties of the coresponding protein to be analyzed
for pp in ppb.build_peptides(structure):
    seq_info = {} # empty dict
    seq = pp.get_sequence() 
    analyzed_seq = ProteinAnalysis(str(seq)) # convert to a str 
    
    # Some examples of furthur analysis of the analyzed_seq object
    # Storing results in a dictionary    
    seq_info['Sequence Number'] = counter # set sequence id
    seq_info['Sequence'] = seq # store BioPython Seq() object
    seq_info['Sequence Length'] = len(seq) # length of seq
    seq_info['Molecular Weight'] = analyzed_seq.molecular_weight() # molecular weight
    seq_info['GRAVY'] = analyzed_seq.gravy() # hydrophobicity 
    seq_info['AA Count'] = analyzed_seq.count_amino_acids() 
    seq_info['AA Percent'] = analyzed_seq.get_amino_acids_percent()
    
    # Update all_seqs list 
    all_seqs.append(seq_info)
    counter += 1

# Print results
for i in all_seqs:
    print(all_seqs[i])

### Pairwise Sequence Alignment

In [ ]:
# Read in file
Fasta = list(SeqIO.parse("allergen_sequence.fasta", "fasta"))

size = len(Fasta)

#Initiate variable i and j for for loop
i = 0
j = 0

#Initiate an empy numpy matrix that is the length of the fasta file
alignments_score = np.empty(shape=(size,size))

#Nested for loop to do pairwise sequence alignment for each pair of kinases
def pairwise_seq_heatmap(my_numpy_array, size):
    for i in range(size):
        for j in range(size):
        # Define two sequences to be aligned
            X = Fasta[i].seq
            Y = Fasta[j].seq
            print(Fasta[i].id)
            
        # Get a list of the global alignments between the two sequences ACGGGT and ACG
        # No parameters. Identical characters have score of 1, else 0.
        # No gap penalties.
            my_numpy_array[i,j] = int(pairwise2.align.globalxx(X, Y, score_only=True))

    return my_numpy_array
# Check that the for loop correctly created the matrix
My_alignment_Scores = pairwise_seq_heatmap(alignments_score, size)

### Heat Plot

In [ ]:
# Create subplot
fig, ax = plt.subplots()

# show the heatplot
plt.imshow(alignments_score)

# Construct the heat map to show the axes
im = ax.imshow(alignments_score)

# Add a colorbar to the right of the heatmap
cbar = ax.figure.colorbar(im)

# Show the plot
plt.show()

### Analysis of the Results

The code did not run but theotically cashew should have a higher alignment score with pistachio and higher structural similarity in the IgE binding epitope.